#### Assignment 2 - Markov Decision Process Replacement Problem

## Stationary Distribution

In [200]:
import numpy as np

# Define the size of the matrix (91x91)
matrix_size = 91

# build transition matrix (P Matrix)
transition_matrix = np.zeros((matrix_size, matrix_size))
transition_matrix[:, 0] = np.round(0.10 + 0.01 * np.arange(matrix_size), 2)
np.fill_diagonal(transition_matrix[:, 1:], np.round(0.90 - 0.01 * np.arange(matrix_size), 2))

# Find left eigen vector with eigen value equal to 1 
eigenvalues, eigenvectors = np.linalg.eig(transition_matrix.T)
eigenvalue_1_index = np.isclose(eigenvalues, 1)
ev = eigenvectors[:, eigenvalue_1_index].flatten()

# normalize
stationary_distribution = ev / ev.sum()

print(transition_matrix)
print(stationary_distribution)

[[0.1  0.9  0.   ... 0.   0.   0.  ]
 [0.11 0.   0.89 ... 0.   0.   0.  ]
 [0.12 0.   0.   ... 0.   0.   0.  ]
 ...
 [0.98 0.   0.   ... 0.   0.02 0.  ]
 [0.99 0.   0.   ... 0.   0.   0.01]
 [1.   0.   0.   ... 0.   0.   0.  ]]
[1.46097542e-01+0.j 1.31487788e-01+0.j 1.17024131e-01+0.j
 1.02981235e-01+0.j 8.95936746e-02+0.j 7.70505602e-02+0.j
 6.54929762e-02+0.j 5.50141000e-02+0.j 4.56617030e-02+0.j
 3.74425964e-02+0.j 3.03285031e-02+0.j 2.42628025e-02+0.j
 1.91676140e-02+0.j 1.49507389e-02+0.j 1.15120690e-02+0.j
 8.74917240e-03+0.j 6.56187930e-03+0.j 4.85579068e-03+0.j
 3.54472720e-03+0.j 2.55220358e-03+0.j 1.81206454e-03+0.j
 1.26844518e-03+0.j 8.75227175e-04+0.j 5.95154479e-04+0.j
 3.98753501e-04+0.j 2.63177311e-04+0.j 1.71065252e-04+0.j
 1.09481761e-04+0.j 6.89735096e-05+0.j 4.27635759e-05+0.j
 2.60857813e-05+0.j 1.56514688e-05+0.j 9.23436658e-06+0.j
 5.35593262e-06+0.j 3.05288159e-06+0.j 1.70961369e-06+0.j
 9.40287531e-07+0.j 5.07755267e-07+0.j 2.69110291e-07+0.j
 1.39937351e-07+0.

## Poisson Equation

In [204]:
import numpy as np

tarr = []
matrix_size = 91
for i in range(matrix_size):
    tr = [0] * (matrix_size)
    # Set the diagonal element
    tr[i] = -round(0.90 - 0.01 * i, 2)

    if i != 0:
        # Set the 1's in the subdiagonal
        tr[i-1] = 1  
    tr[-1] = 1

    tarr.append(tr)
    
a = np.array(tarr)
# b = np.array([1] + [0] * (matrix_size-1))
b = 0.10 + 0.01 * np.arange(matrix_size) 

solution = np.linalg.solve(a, b)
print(b)
print(solution)

[0.1  0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23
 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37
 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51
 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65
 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79
 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93
 0.94 0.95 0.96 0.97 0.98 0.99 1.  ]
[0.05121949 0.09810903 0.14114383 0.1807372  0.2172497  0.25099676
 0.28225512 0.31126827 0.33825099 0.36339325 0.38686349 0.40881143
 0.42937048 0.44865976 0.46678593 0.48384463 0.49992185 0.51509506
 0.52943416 0.5430024  0.55585706 0.56805015 0.57962896 0.59063657
 0.60111229 0.61109205 0.62060874 0.6296925  0.63837104 0.64666981
 0.65461225 0.66221999 0.66951298 0.67650969 0.6832272  0.68968134
 0.69588683 0.7018573  0.70760546 0.71314314 0.71848136 0.72363041
 0.72859991 0.73339883 0.73803559 0.74251806 0.74685364 0.75104926
 

### Policy Iteration

In [205]:
max_iterations = 20
count = 0
terminate = False
matrix_size = 91
# 0. Fix a policy
policy = np.array([0] * (matrix_size))
poisson_a = np.copy(a)
poisson_b = 0.10 + 0.01 * np.arange(matrix_size)
print(poisson_b)
while terminate == False and count < max_iterations:
    # 1. Find a solution for poisson equation (eval)
    for i in range(len(policy)):
        if policy[i] == 1:
            poisson_b[i] = 0.6
            poisson_a[i][i] = 0.0
    
    new_val = np.linalg.solve(poisson_a, poisson_b)
    
    # 2. Determine argmax (min)
    a2 = 0.6
    new_policy = []
    for i in range(len(new_val)):
        a1 = transition_matrix[i][0] * 1 + (1-transition_matrix[i][0]) * new_val[i]
        argmin = np.argmin([a1, a2])
        new_policy.append(argmin)
        
    
    new_policy_np = np.array(new_policy)
    
    # 3. Check if policies are the same
    if np.array_equal(policy, new_policy_np):
        # print('done')
        break
    else:
        policy = new_policy_np 
        count += 1
    
print(new_policy_np)
print(new_val)
print(count)



[0.1  0.11 0.12 0.13 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23
 0.24 0.25 0.26 0.27 0.28 0.29 0.3  0.31 0.32 0.33 0.34 0.35 0.36 0.37
 0.38 0.39 0.4  0.41 0.42 0.43 0.44 0.45 0.46 0.47 0.48 0.49 0.5  0.51
 0.52 0.53 0.54 0.55 0.56 0.57 0.58 0.59 0.6  0.61 0.62 0.63 0.64 0.65
 0.66 0.67 0.68 0.69 0.7  0.71 0.72 0.73 0.74 0.75 0.76 0.77 0.78 0.79
 0.8  0.81 0.82 0.83 0.84 0.85 0.86 0.87 0.88 0.89 0.9  0.91 0.92 0.93
 0.94 0.95 0.96 0.97 0.98 0.99 1.  ]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
[0.05109127 0.0978353  0.14070165 0.18009632 0.21637031 0.24982642
 0.28072448 0.30928509 0.33569176 0.36009124 0.38259173 0.40325807
 0.42210285 0.43907142 0.45401785 0.45401785 0.45401785 0.45401785
 0.45401785 0.45401785 0.45401785 0.45401785 0.45401785 0.45401785
 0.45401785 0.45401785 0.45401785 0.45401785 0.45401785 0.45401785
 0.45401785 0.45

## Value Iteration

In [196]:
V_prev = np.array([0.0] * 91)
p_matrix = np.copy(transition_matrix)
count = 0
policy = np.array([0.0] * 91)
while True:
    V_new = np.array([0.0] * 91)
    for i in range(len(V_prev)):
        if i == 90:
            a1 = V_prev[0] + 1  
        else:  
            a1 = (1-p_matrix[i][0]) * V_prev[i+1] + (p_matrix[i][0] * (1 + V_prev[0]))
        a2 = V_prev[0] + 0.6
        V_new[i] = np.min([a1, a2])
        policy[i] = np.argmin([a1, a2])
    
    diff = V_new - V_prev
    span = np.max(diff) - np.min(diff)
    
    if span < 0.0000001:
        break
    V_prev = V_new.copy() - np.max(V_new)
    count += 1
    
    
print(V_new)
print(policy)

[-0.3082571  -0.25728885 -0.21068444 -0.16797975 -0.12877577 -0.09273066
 -0.05955375 -0.02900142 -0.00087505  0.02497883  0.04866554  0.07023552
  0.08967213  0.10687201  0.1216138   0.13350919  0.14192658  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146  0.14587146
  0.14587146  0.14587146  0.14587146  0.14587146  0